<a href="https://colab.research.google.com/github/machiwao/CCTHESS1-CCTHESS2-Dev-and-Docs/blob/coli/ERA5_Land_Daily_Aggregated_'14_'23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install earthengine-api --upgrade
# optional, but useful for interactive maps
!pip install geemap --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.2/463.2 kB 8.9 MB/s eta 0:00:00
  Attempting uninstall: earthengine-api
    Found existing installation: earthengine-api 1.5.24
    Uninstalling earthengine-api-1.5.24:
      Successfully uninstalled earthengine-api-1.5.24
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 631.5/631.5 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 76.5 MB/s eta 0:00:00
  Attempting uninstall: geemap
    Found existing installation: geemap 0.35.3
    Uninstalling geemap-0.35.3:
      Successfully uninstalled geemap-0.35.3


# New Import Section


In [2]:
import ee, pandas as pd
import geemap

# Authenticate

In [3]:
ee.Authenticate()
ee.Initialize(project='data-scrapping-470316') #name ng project niyo sa earth engine

# click niyo yung left side ng malapit sa dp
# tapos project info
# then under Cloud Project ayun i-copy paste


*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_7TDKVSyKvBdmMqW?ref=4i2o6


# Load ERA5-Land Daily Aggregated


In [26]:
# ERA5-Land Daily Aggregates dataset
dataset = ee.ImageCollection("ECMWF/ERA5_LAND/DAILY_AGGR") \
  .filterDate('2014-01-01', '2023-12-31')

# Print The List of Bands


In [27]:
# Print available variables (bands)
bands = dataset.first().bandNames().getInfo()
print("Available Bands:", bands)

Available Bands: ['dewpoint_temperature_2m', 'temperature_2m', 'skin_temperature', 'soil_temperature_level_1', 'soil_temperature_level_2', 'soil_temperature_level_3', 'soil_temperature_level_4', 'lake_bottom_temperature', 'lake_ice_depth', 'lake_ice_temperature', 'lake_mix_layer_depth', 'lake_mix_layer_temperature', 'lake_shape_factor', 'lake_total_layer_temperature', 'snow_albedo', 'snow_cover', 'snow_density', 'snow_depth', 'snow_depth_water_equivalent', 'snowfall_sum', 'snowmelt_sum', 'temperature_of_snow_layer', 'skin_reservoir_content', 'volumetric_soil_water_layer_1', 'volumetric_soil_water_layer_2', 'volumetric_soil_water_layer_3', 'volumetric_soil_water_layer_4', 'forecast_albedo', 'surface_latent_heat_flux_sum', 'surface_net_solar_radiation_sum', 'surface_net_thermal_radiation_sum', 'surface_sensible_heat_flux_sum', 'surface_solar_radiation_downwards_sum', 'surface_thermal_radiation_downwards_sum', 'evaporation_from_bare_soil_sum', 'evaporation_from_open_water_surfaces_excludi

# Aparri Location

In [28]:
# Geometry Point
aparri = ee.Geometry.Point([18.3342, 121.6055]).buffer(25000)

# Extract Daily Data


In [29]:
def extract(img):
    vals = img.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=aparri,
        scale=10000,
        maxPixels=1e13
    )
    return ee.Feature(None, vals).set("date", img.date().format("YYYY-MM-dd"))

fc = dataset.map(extract)


# Fetch all bands + values

In [30]:
dates = fc.aggregate_array("date").getInfo()
var_names = dataset.first().bandNames().getInfo()
data = {v: fc.aggregate_array(v).getInfo() for v in var_names}

# Build Dataframe

In [31]:
df = pd.DataFrame({"date": dates})
for v in var_names:
    df[v] = data[v]

/tmp/ipython-input-3832312120.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[v] = data[v]
/tmp/ipython-input-3832312120.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[v] = data[v]
/tmp/ipython-input-3832312120.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[v] = data[v]
/tmp/ip

# Units Metadata + Concversions

In [32]:
metadata = []

for col in df.columns:
    if col == "date":
        continue

    original_name = col
    original_unit = "varies"
    converted_unit = original_unit
    new_name = col

    # Kelvin → Celsius
    if "temperature" in col:
        df[col] = df[col] - 273.15
        new_name = col + "_C"
        original_unit, converted_unit = "K", "°C"

    # Precipitation / Runoff / Evaporation (m → mm/day)
    elif col.endswith("_sum") or "precipitation" in col or "evaporation" in col or "runoff" in col:
        df[col] = df[col] * 1000
        new_name = col + "_mm"
        original_unit, converted_unit = "m", "mm/day"

    # Pressure (Pa → hPa)
    elif "pressure" in col:
        df[col] = df[col] / 100
        new_name = col + "_hPa"
        original_unit, converted_unit = "Pa", "hPa"

    # Wind (m/s → km/h)
    elif "u_component" in col or "v_component" in col or "wind" in col:
        df[col] = df[col] * 3.6
        new_name = col + "_kmh"
        original_unit, converted_unit = "m/s", "km/h"

    # Snow depth (m → cm)
    elif "snow_depth" in col:
        df[col] = df[col] * 100
        new_name = col + "_cm"
        original_unit, converted_unit = "m", "cm"

    # Radiation & Fluxes (J/m²/day → W/m²)
    elif col.endswith("_radiation_sum") or col.endswith("_flux_sum") or col.endswith("_heat_sum"):
        df[col] = df[col] / 86400
        new_name = col + "_Wm2"
        original_unit, converted_unit = "J/m²/day", "W/m²"

    # Rename
    df.rename(columns={col: new_name}, inplace=True)

    # Add metadata record
    metadata.append({
        "Band": original_name,
        "Converted Column": new_name,
        "Original Unit": original_unit,
        "Converted Unit": converted_unit
    })

In [33]:
# Build metadata DataFrame
meta_df = pd.DataFrame(metadata)

# Download CSV

In [34]:
from google.colab import files

filename = "ERA5-Land Daily Aggregated_2014_2023_Aparri_1.1.csv"
df.to_csv(filename, index=False)

# --- Download ---
files.download(filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#

In [35]:
print(df.head())

         date  dewpoint_temperature_2m_C  temperature_2m_C  \
0  2014-01-01                  -3.879676         -2.498737   
1  2014-01-02                  -2.082665         -0.609773   
2  2014-01-03                  -0.333795          1.465666   
3  2014-01-04                  -1.897434         -0.374894   
4  2014-01-05                  -0.245307          0.955377   

   skin_temperature_C  soil_temperature_level_1_C  soil_temperature_level_2_C  \
0           -2.921936                   -1.673869                   -1.682923   
1           -1.765258                   -1.728308                   -1.656659   
2            0.143430                   -1.575775                   -1.615431   
3           -1.641404                   -1.553010                   -1.560497   
4            0.233687                   -1.468951                   -1.516594   

   soil_temperature_level_3_C  soil_temperature_level_4_C  \
0                   -1.261921                    1.087510   
1                 